<a href="https://colab.research.google.com/github/someshsingh22/Sherlocked/blob/master/Sherlocked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
! git clone https://github.com/someshsingh22/Sherlocked

Cloning into 'Sherlocked'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 60 (delta 18), reused 38 (delta 1), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import Counter
import os
import re
from argparse import Namespace
import time
from tqdm import tqdm

In [0]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    return ' '.join(words).encode('utf-8')


In [0]:
 y=predict(device, net, "Sherlock Holmes".split(), n_vocab, vocab_to_int, int_to_vocab, top_k=2)

In [0]:
class Pen :
  def __init__(self,flags,brain):
    self.Brain=brain
    self.Network=self.Brain.Network
    self.Data=self.Brain.Data
    self.Optimizer=self.Brain.Optimizer
    self.flags=flags
    
    # Temperatur Randomness
    def temp_choice(temp):
      probs=np.asarray([80-60*temp,60-20*temp,50,40+20*temp,60+20*temp])
      return probs/250
    
    # Writer
    def writer(self,words="I am",temperature=0.5,length=70):
      self.Network.eval()
      state_h, state_c = self.Network.zero_state(1)
      state_h = state_h.to(self.flags.device)
      state_c = state_c.to(self.flags.device)
      
      for w in words.split():
        ix = torch.tensor([[self.Data.vocab_to_int[w]]]).to(self.flags.device)
        output, (state_h, state_c) = self.Network(ix, (state_h, state_c))
        top_k=np.random.choice(np.arange(1,6),1,p=temp_choice(temperature))
     
      _, top_ix = torch.topk(output[0], k=top_k)
      choices = top_ix.tolist()
      choice = np.random.choice(choices[0])
      words.append(self.Data.int_to_vocab[choice])
      
      for _ in range(length):
          ix = torch.tensor([[choice]]).to(self.flags.device)
          output, (state_h, state_c) = self.Network(ix, (state_h, state_c))
          top_k=np.random.choice(np.arange(1,6),1,p=temp_choice(temperature))
          
          _, top_ix = torch.topk(output[0], k=top_k)
          choices = top_ix.tolist()
          choice = np.random.choice(choices[0])
          words.append(self.Data.int_to_vocab[choice])

      print(' '.join(words))
      return ' '.join(words)

In [0]:
pen=Pen(Pen_flags,br)

In [0]:
class Arthur :
  def __init__(self,epochs=50):
    self.Data=Data(Data_flags)
    self.Network=Network(Brain_flags,Data)
    self.Optimizer=Optimizer(self.Network)
    self.Brain=Brain(Brain_flags,Optimizer)
    self.epochs=epochs
    self.Pen=Pen(Pen_flags,Brain)
    self.models=[]
    self.output=[]
    
    # Train
    def Train(self):
      self.Brain.Train(self.epochs)
    
    # Write
    def Write(self):
      output.append(self.Pen.Write())
      models.append(self.Brain.flags)
    
    

In [0]:
pen.writer()

AttributeError: ignored

In [0]:
class Arthur:
  def __init__(self,flags):
    self.flags=flags
    self.Data=Data(flags)
    self.Brain=Brain(flags,self.Data.n_vocab)    

In [0]:
# Data Class
class Data :
  def __init__(self,flags):
    self.flags=flags
    
    # Pre Process Data
    self.pre_process()
    
  # Prepare Data For Training
  def pre_process(self):
    start=time.time()
    # Read datat and split to words from files
    text=open(self.flags.data_dir).read().split()

    # Create Frequency Dictionary
    word_counts = Counter(text)
    self.sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)

    # Word2Vec Mapping
    self.int_to_vocab = {k: w for k, w in enumerate(self.sorted_vocab)}
    self.vocab_to_int = {w: k for k, w in self.int_to_vocab.items()}
    self.n_vocab = len(self.int_to_vocab)

    # Create Input-Output Data
    int_text = [self.vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (self.flags.seq_size * self.flags.batch_size))
    in_text = int_text[:num_batches * self.flags.batch_size * self.flags.seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (Data_flags.batch_size, -1))
    out_text = np.reshape(out_text, (Data_flags.batch_size, -1))
    print("Data Preprocessing complete with {} words".format(self.n_vocab))
    self.in_text=in_text
    self.out_text=out_text

  # Create Input-Output Batch Generator 
  def get_batches(self):
    num_batches = np.prod(self.in_text.shape) // (self.flags.seq_size * self.flags.batch_size)
    for i in range(0, num_batches * self.flags.seq_size, self.flags.seq_size):
      yield self.in_text[:, i:i+self.flags.seq_size], self.out_text[:, i:i+self.flags.seq_size]

In [0]:
class Brain(nn.Module):
  def __init__(self,flags,n_vocab):
    super(Brain,self).__init__()
    self.flags=flags
    self.embedding= nn.Embedding(n_vocab, flags.embedding_size)
    self.lstm=nn.LSTM(flags.embedding_size, flags.lstm_size, batch_first=True, num_layers=flags.num_layers, dropout=flags.dropout)
    self.dense = nn.Linear(flags.lstm_size, n_vocab)
    self.criterion = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.Adam(self.parameters(), lr=self.flags.lr)
    self.loss_value=0
    self.to(flags.device)
    
  # Forward Pass
  def forward(self, x, prev_state):
      embed = self.embedding(x)
      output, state = self.lstm(embed, prev_state)
      logits = self.dense(output)
      return logits, state

  # ZeroState Init
  def zero_state(self,batch_size):
        return (torch.zeros(self.flags.num_layers*(2 if self.flags.is_bidirectional else 1), batch_size, self.flags.lstm_size),
                torch.zeros(self.flags.num_layers*(2 if self.flags.is_bidirectional else 1), batch_size, self.flags.lstm_size))
  
  #Train Function
  def Train(self,epochs,Data):
    self.train()
    for epoch in range(epochs):
      start=time.time()
      batches=Data.get_batches()
      state_h, state_c = self.zero_state(self.flags.lstm_size)
      
      
      # Transfer data to GPU
      state_h = state_h.to(self.flags.device)
      state_c = state_c.to(self.flags.device)
      
      for x, y in tqdm(batches):
        
        # Reset all gradients
        self.optimizer.zero_grad()

        # Transfer data to GPU
        x = torch.tensor(x).to(self.flags.device)
        y = torch.tensor(y).to(self.flags.device)

        logits, (state_h, state_c) = self(x, (state_h, state_c))
        self.loss = self.criterion(logits.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        self.loss_value = self.loss.item()
        
        # Update the network's parameters
        self.optimizer.step()
        self.loss.backward()
        _ = torch.nn.utils.clip_grad_norm_(self.parameters(), self.flags.gradients_norm)
        #self.Optimizer.decay(self.loss)
        
        self.optimizer.step()
      print('Epoch: {}/{}'.format(epoch, epochs),'Loss: {}'.format(self.loss_value),'Time Taken : {}'.format(time.time()-start))
  
  

In [67]:
art=Arthur(flags)

Data Preprocessing complete with 21252 words


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [68]:
art.Brain.Train(2,art.Data)

7it [00:02,  2.54it/s]


KeyboardInterrupt: ignored

46it [00:18,  2.44it/s]
0it [00:00, ?it/s]

Epoch: 0/2 Loss: 5.393995761871338 Time Taken : 18.442371129989624


46it [00:18,  2.42it/s]

Epoch: 1/2 Loss: 4.833022117614746 Time Taken : 18.931352853775024
